In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

# 구글 드라이브 파일 확인
!ls '/gdrive/My Drive/GJ_AI_Colab/프로젝트/data/'

# 반복되는 드라이브 경로 변수화
drive_path = '/gdrive/My Drive/GJ_AI_Colab/프로젝트/data/'

In [ ]:
!git clone https://github.com/ssut/py-hanspell.git
%cd /content/py-hanspell
!python setup.py install

In [ ]:
!pip install pickle5
!pip3 install konlpy  # 한국어 형태소 분석을 위해

In [ ]:
import pandas as pd
import pickle5 as pickle
import re
import nltk

from hanspell import spell_checker
from konlpy.tag import Okt
from nltk.corpus import stopwords

In [ ]:
original_df = pd.read_pickle(drive_path + 'original_data.pkl')

In [ ]:
# 전처리 - okt 이용
def okt_preprocessing(name_list):
    stopword_list = ['할인', '재구매', '입점', '발송', '순차', '연휴', '황금', '출시', '신제품', '신상', 
                 '이벤트', '택배', '예약', '출고', '생일', '선물', '답례품', '증정', '당일', '무료', 
                 '배송', '무배', '주문', '1위', '판매', '폭주', '재입고', '입고']

    nltk.download('stopwords')
    nltk.download('punkt')
    stop_words = set(stopwords.words('english'))

    okt = Okt()
    okt_name = []

    for sent in name_list:
        sent = re.sub('[^\w\s./]', ' ', sent)
        for sw in stopword_list:
            sent = sent.replace(sw, ' ')  
        sent = okt.morphs(sent)
        result = ''
        for token in sent:
            if token not in stop_words:
                result += ' ' + token
        sent = re.sub('[\s]+', ' ', result) # 연속된 공백 문자 하나로 변경
        sent = sent.strip() # 앞뒤 공백 제거
        okt_name.append(sent)
    
    return okt_name

In [ ]:
rename = okt_preprocessing(original_df['name'])

text_preprocessing_df = pd.DataFrame({'id': original_df['id'], 'name': rename, 'label': original_df['label']})
text_preprocessing_df.to_pickle(drive_path + 'okt_data.pkl')

In [ ]:
# 전처리 - py-hanspell 이용
def pyhanspell_preprocessing(name_list):
    stopword_list = ['할인', '재구매', '입점', '발송', '순차', '연휴', '황금', '출시', '신제품', '신상', 
                 '이벤트', '택배', '예약', '출고', '생일', '선물', '답례품', '증정', '당일', '무료', 
                 '배송', '무배', '주문', '1위', '판매', '폭주', '재입고', '입고']

    pyhanspell_name = []

    for sent in name_list:
        sent = re.sub('[^\w\s./]', ' ', sent)
        for sw in stopword_list:
            sent = sent.replace(sw, ' ')
        sent = re.sub('[\s]+', ' ',sent) # 연속된 공백 문자 하나로 변경
        sent = sent.strip() # 앞뒤 공백 제거
        spelled_sent = spell_checker.check(sent)
        checked_sent = spelled_sent.checked
        pyhanspell_name.append(checked_sent)

    return pyhanspell_name

In [ ]:
rename = pyhanspell_preprocessing(original_df['name'])

text_preprocessing_df = pd.DataFrame({'id': original_df['id'], 'name': rename, 'label': original_df['label']})
text_preprocessing_df.to_pickle(drive_path + 'pyhanspell_data.pkl')